In [11]:
import pickle
from estnltk import Text
from tqdm import tqdm
from collections import defaultdict
from conllu import parse, parse_tree
from random import shuffle

In [2]:
with open("../../train_1.conllu", "r") as fin:
    data = fin.read()

In [3]:
sentences2 = parse(data)

In [ ]:
# mida ei saa asendada:
#+ ka ei saa olla asendatav ega asendaja => tokenlist? filter? changed etc
#+ vpart alluvaga verb => verbi lapse deprel vpart
#+ verbiahelas olev 1. verb => verbi laps on verb
#+ olema kui on PRD => olema, kui ta laps on prd
#+ asenduseks pandav sõna ei saa olla "olema", kui verbi alluv OBJ
#+ <Q alluvaga määrsõna/nimisõna?/arvsõna? kui laps on <Q, siis ei saa
#+ vpart ei saa asendada, kui sõna deprel vpart
# määrsõnale või omadussõnale alluv määrsõna -> kui määrsõna/omadussõna laps on määrsõna,siis seda ei saa asendada


In [4]:
bad_subs = defaultdict(list)
good_subs = []
for sent in tqdm(sentences2):
    
    changed_token = sent.filter(deps = 'CHANGED')[0]
    misc = list(changed_token['misc'].keys())[0]
    misc_lemmas = Text(misc).tag_layer().lemma[0]

    if changed_token['upos'] == 'ka' or 'ka' in misc_lemmas:
        bad_subs[1].append(sent)
        continue
    
    if changed_token['deprel'] == '@Vpart':    
        bad_subs[2].append(sent)
        continue

    if changed_token['lemma'] == 'olema':
        objects = sent.filter(deprel = '@OBJ', head = changed_token['id'])
        if objects:
            bad_subs[3].append(sent)
            continue
        
    if 'olema' in misc_lemmas:
        predicatives = sent.filter(deprel = '@PRD', head = changed_token['id'])
        if predicatives:
            bad_subs[4].append(sent)
            continue
                    
    if changed_token['upos'] == 'V':       
        verb_children = sent.filter(upos = 'V', head = changed_token['id'])
        verb_parent = sent.filter(upos = 'V', id = changed_token['head'])
        sup_inf_children = False
        for ch in verb_children:
            if ch['feats']:
                if 'sup' in ch['feats'] or 'inf' in ch['feats']:
                    sup_inf_children = True
                    break
        if sup_inf_children:
            bad_subs[5].append(sent)
            continue
        
        if verb_parent:
            if 'sup' in verb_parent[0]['feats'] or 'inf' in verb_parent[0]['feats']:
                bad_subs[5].append(sent)
                continue
        
        vpart_children = sent.filter(deprel = '@Vpart', head = changed_token['id'])
        if vpart_children:
            bad_subs[6].append(sent)
            continue
    
    if changed_token['upos'] != 'N':
        q_children = sent.filter(deprel = '@<Q', head = changed_token['id'])
        q_children2 = sent.filter(deprel = '@Q>', head = changed_token['id'])
        if q_children or q_children2:
            bad_subs[7].append(sent)
            continue
        
    if changed_token['upos'] == 'A' or changed_token['upos'] == 'D':
        adverb_children = sent.filter(upos = 'D', head = changed_token['id'])
        if adverb_children:
            bad_subs[8].append(sent)
            continue
    
    good_subs.append(sent)

100%|██████████| 103581/103581 [03:17<00:00, 525.70it/s]


In [5]:
for k, v in bad_subs.items():
    print(k, len(v))

3 417
7 268
5 2399
4 2186
2 1035
6 1203
8 1290
1 494


In [6]:
494+1290+1203+1035+2186+2399+268+417

9292

In [7]:
len(good_subs)

94289

In [8]:
9292/(9292+94289)

0.08970757185198057